In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
# Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Precprocess the data
# Drop irrelevant features row num, cust id, surname
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encode categorical variable Gender
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [5]:
# One Hot Encode Geography column(it can't be label encoded like France-0, Spain-1, Germany-2 bcz the model will consider germany>spain and france because it is labelled as 2)

from sklearn.preprocessing import OneHotEncoder
oneHotencode_geography= OneHotEncoder()
geo_encoder=oneHotencode_geography.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
oneHotencode_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=oneHotencode_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combine one hot encoded columns with original data

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Save the encoders and scalar
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('oneHotencode_geography.pkl','wb') as file:
    pickle.dump(oneHotencode_geography,file)


In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## Divide the dataset into dependent and independent features
# Exited- dependent feature, and remaining columns- independent feature

X=data.drop('Exited', axis=1)
y=data['Exited']

# Split the data into training and test data
X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.2, random_state=42)

# Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [13]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


ANN IMPLEMENTATION

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
(X_train.shape[1],)

(12,)

In [18]:
# Build ANN model
model=Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),  #First hidden layer connected with input layer
    Dense(32, activation='relu'), # hidden layer 2
    Dense(1, activation='sigmoid')  # output layer    
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
import tensorflow
# define the optimizer and loss
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [21]:
## Compile the model (fwd and back prop)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [22]:
## Setup the TENSORBOARD
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_cbk=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
# Setup Early Stopping
early_stopping_cbk= EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [24]:
### TRAINING THE MODEL

history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks=[tensorflow_cbk,early_stopping_cbk]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.3935 - accuracy: 0.8406 - val_loss: 0.3595 - val_accuracy: 0.8595
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3544 - accuracy: 0.8575 - val_loss: 0.3409 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3444 - accuracy: 0.8584 - val_loss: 0.3428 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3435 - accuracy: 0.8601 - val_loss: 0.3521 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3397 - accuracy: 0.8605 - val_loss: 0.3448 - val_accuracy: 0.8605
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3384 - accuracy: 0.8609 - val_loss: 0.3460 - val_accuracy: 0.8505
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3329 - accuracy: 0.8630 - val_loss: 0.3449 - val_accuracy: 0.85

In [25]:
model.save('model.h5')

c:\Users\aasth\UDEMY PROJECTS\GEN_AI_KRISHNAIK\ANN_ChurnPredictionn\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
## Load Tensorboard Extension

%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit